In [1]:
print("hello")

hello


In [2]:
# @title Pip install comands
!pip install librosa
!pip install torch torchvision torchaudio
!pip install torch
!pip install jiwer
!pip install wandb
!pip install datasets
!pip install kaggle

Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105787 sha256=33faa3b54e4685361e9a4fdd3aa34542b17cb6fd527495164b3e6cade63d9d8b
  Stored in directory: /Users/nidalhulaihel/Library/Caches/pip/wheels/41/a6/66/4da54ffa475d3c5249032689dd1d051776c852d39d40b901ee
Successfully built kaggle


In [3]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification
!unzip gtzan-dataset-music-genre-classification.zip

Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
License(s): other
100%|█████████████████████████████████████▉| 1.21G/1.21G [00:58<00:00, 23.0MB/s]
100%|██████████████████████████████████████| 1.21G/1.21G [00:58<00:00, 22.2MB/s]


In [12]:
import os
import torchaudio
import torch
from torch.utils.data import DataLoader, Dataset

# Custom dataset class
class GTZANDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_paths = []
        self.labels = []
        self.genres = os.listdir(root_dir)
        
        for idx, genre in enumerate(self.genres):
            genre_dir = os.path.join(self.root_dir, genre)
            for file_name in os.listdir(genre_dir):
                file_path = os.path.join(genre_dir, file_name)
                self.file_paths.append(file_path)
                self.labels.append(idx)  # Use the genre index as the label

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        waveform, sample_rate = torchaudio.load(file_path)
        label = torch.tensor(self.labels[idx])

        # Apply any transformation if provided
        if self.transform:
            waveform = self.transform(waveform)
        
        return waveform, label

# Collate function for padding
def collate_fn(batch):
    waveforms = [item[0].squeeze(0) for item in batch] 
    labels = [item[1] for item in batch]
    
    waveforms = torch.nn.utils.rnn.pad_sequence(waveforms, batch_first=True)
    labels = torch.tensor(labels)
    
    return waveforms, labels



Waveforms shape: torch.Size([16, 661794]), Labels: tensor([5, 9, 2, 9, 9, 5, 0, 5, 4, 8, 5, 8, 0, 4, 8, 8])


In [14]:
dataset = GTZANDataset(root_dir="data/genres_original/")
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

# Testing the DataLoader
for batch in dataloader:
    waveforms, labels = batch
    print(f"Waveforms shape: {waveforms.shape}, Labels: {labels}")
    break

Waveforms shape: torch.Size([16, 675808]), Labels: tensor([7, 3, 9, 4, 6, 6, 7, 1, 3, 5, 5, 2, 8, 3, 9, 0])
